### Eventloop
Ein Eventloop ist eine Schleife, die z.B. all 100 Millisekunden prüft, ob
neue Events ( Tastendrücke, Mausklicks, Mausbewegungen, ...) stattgefunden haben und dann
entsprechend reagiert.

Wir benutzen einen Eventloop, um mir den Pfeiltasten eine Linie zu zeichnen.
Die Linie setzt sich jede Sekunde jeweils ein paar Pixel in
Richtung der zuletzt gedrücken Pfeiltaste fort.

Wird eine Pfeiltaste gedrückt, wird diese als erstes Element in eine Liste
`event_queue` eingefügt. 
Ist der Eventqueue  nicht leer, entfernt 
der Eventloop die zuletzt gedrückt Taste, andernfalls wird
die zuletzt gedrückt Taste beibehalten. Die Taste wird in der Variable `key` gespeichert.
Dann wird die Funktione `move` mit den Argumenten `key_move[key]` aufgerufen,
welche die Linie auf der Leinwand fortsetzt. 

Der Event-Loop wird mit Hilfe der Funktion `threading.Timer`.
Nachstehend definierte Funktion `event_loop` 
macht sich zu einem Thread, der mit etwas Verzögerung startet.
Und dieser Thread macht aus der Funktion erneut einen solchen Thread.
Dieser Vorgang wird gestoppt, wenn das `stop_event` gesetzt wird.


```python
def move(dx, dy):
    '''zeichen Linie von der aktuellen Position (x, y) 
       nach (x+dx, y+dy)
    '''
    
event_queue = ['ArrowUp', 'ArrowRight']

key_move = {'ArrowUp': (0, -5),
            'ArrowDown': (0, 5),
            'ArrowLeft': (-5, 0),
            'ArrowRight': (5, 0),
            }
            
def event_loop(last_event=None, count=0):
    if event_queue:
        key = event_queue.pop()
    else:
        key = last_event

    if key in key_move:
        dx, dy = key_move[key]
        move(dx, dy)

    if not stop_event.is_set():
        thread = threading.Timer(1, event_loop, args=(key, count+1))
        thread.name = f'Eventloop-{count}'
        thread.start()            
```

***
**threading.Event**:  
Ein `threading.Event` ist im wesentlichen ein Flag, das entweder `True` oder `False` ist.
Es hat folgende Methoden:
- `stop_event = threading.Event()`: Erstellt ein `threading.Event`,
- `is_set()` : liefert `True` oder `False` (Default),
- `set()`    : `is_set()` liefert nun `True`,
- `clear()`  : `is_set()` liefert nun `False`.
***

In [ ]:
import threading


stop_event = threading.Event()
stop_event.is_set()

In [ ]:
stop_event.set()
stop_event.is_set()

In [ ]:
stop_event.clear()
stop_event.is_set()

In [3]:
import threading
from ipywidgets import Output, Button
from ipycanvas import Canvas
from IPython.display import display


layout = {'border': '1px solid black'}
out = Output(layout=layout)
canvas = Canvas(width=100, height=100, layout=layout)
button_stop = Button(description='Stop')

state = {'pos': (50, 50)}
key_direction = {'ArrowUp': (0, -5),
                 'ArrowDown': (0, 5),
                 'ArrowLeft': (-5, 0),
                 'ArrowRight': (5, 0),
                 }

stop_event = threading.Event()
event_queue = []


@out.capture()
def on_key_down(key, *flags):
    print(key)
    if key in key_direction:
        direction = key_direction[key]
        event_queue.insert(0, direction)


def move(dx, dy):
    x0, y0 = state['pos']
    x, y = (x0+dx, y0+dy)

    canvas.stroke_lines([(x0, y0), (x, y)])
    state['pos'] = (x, y)

    if x < 0 or y < 0 or x > 100 or y > 100:
        canvas.fill_rect(0, 0, canvas.width, canvas.height)
        stop_event.set()


def event_loop(last_event=None, count=0):
    if event_queue:
        direction = event_queue.pop()
    else:
        direction = last_event

    move(*direction)

    if not stop_event.is_set():
        thread = threading.Timer(1, event_loop, args=(direction, count+1))
        thread.name = f'Eventloop-{count}'
        thread.start()


canvas.on_key_down(on_key_down)
button_stop.on_click(lambda bt: stop_event.set())
display(canvas, button_stop, out)

Canvas(height=100, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right=…

Button(description='Stop', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [5]:
event_queue.insert(0, (0, 5))
event_loop()
canvas.focus()  # Canvas-Widget hoert auf Events, ohne das es zuerst angeklickt werden muss

In [ ]:
# lauft der Eventloop Thred noch?
[thread.name for thread in threading.enumerate()[8:]]

In [ ]:
stop_event.clear()
event_loop()

### Aufgabe
Erstellen einen weitere Button `New Game`. Beim Klicken auf diesen Button soll eine 
Funktion `new_game()` aufgerufen werden, welche
- die Leinwand löscht,
- `stop_event.clear()` aufruft,
- den Event-Queue gelöscht,
- `'Right'` in Event-Queue einfügt,
- den Event-Loop startet und der Leinwand Fokus gibt.  

Leite Fehlermeldungen der Funktion `new_game()` ins Output-Widget `out` um.